Note my python version:

In [1]:
import sys
print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


Packages you'll need:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

# Scatter plot example

In [3]:
def sampleBVN(rho, n=50):
    mean = [0, 0]
    cov = [[1, rho], [rho, 1]]
    bvn = np.random.multivariate_normal(mean, cov, n)
    bvn = np.round(bvn, 1) * 10 + 30
    bvn[bvn < 0] = 0
    bvn[bvn > 59] = 59
    r, p_val = st.pearsonr(bvn[:, 0], bvn[:, 1])
    sig = p_val < 0.05
    return bvn, r, sig

In [4]:
N = 1000
sig_rhos = pd.DataFrame({'rho': np.repeat(np.concatenate((np.linspace(-9, -2, num=8), np.linspace(2, 9, num=8)))/10, N)})
apply_sampleBVN_to_rho = list(zip(*sig_rhos['rho'].apply(sampleBVN)))
sig_rhos['bvn'] = list(apply_sampleBVN_to_rho[0])
sig_rhos['r'] = apply_sampleBVN_to_rho[1]
sig_rhos['sig'] = apply_sampleBVN_to_rho[2]
print(sig_rhos.shape)
sig_rhos.head()

(16000, 4)


,rho,bvn,r,sig
0,-0.9,"[[28.0, 32.0], [43.0, 15.0], [31.0, 32.0], [34...",-0.899748,True
1,-0.9,"[[41.0, 18.0], [40.0, 15.0], [33.0, 33.0], [21...",-0.939116,True
2,-0.9,"[[23.0, 41.0], [24.0, 31.0], [30.0, 31.0], [29...",-0.857304,True
3,-0.9,"[[19.0, 35.0], [29.0, 31.0], [20.0, 45.0], [7....",-0.900479,True
4,-0.9,"[[42.0, 13.0], [44.0, 11.0], [28.0, 25.0], [18...",-0.866328,True
